<a href="https://colab.research.google.com/github/tulio-cimat/recomendador-multimodal/blob/main/recuperador_cultural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install essentia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 35.9 MB/s eta 0:00:00


In [20]:
import pandas as pd
import pickle as pkl
import numpy as np
import torch
import tensorflow.keras.applications as transfer
import keras
import requests
import re
import torch.nn as nn
import cv2
from keras import layers
import librosa,librosa.display
import essentia
from essentia.standard import *
import torch
import torch.nn as nn
import os
import IPython

## Funciones auxiliares

In [4]:


#Capas convolución 2d y convolución a embedding
class Conv_2d(nn.Module):
  def __init__(self, input_channels, output_channels, shape=3, pooling=2):
    super(Conv_2d, self).__init__()
    self.conv = nn.Conv2d(input_channels, output_channels, shape, padding=shape//2)
    self.bn = nn.BatchNorm2d(output_channels)
    self.relu = nn.ReLU()
    self.mp = nn.MaxPool2d(pooling)

  def forward(self, x):
    out = self.mp(self.relu(self.bn(self.conv(x))))
    return out

class Conv_emb(nn.Module):
  def __init__(self, input_channels, output_channels):
    super(Conv_emb, self).__init__()
    self.conv = nn.Conv2d(input_channels, output_channels, 1)
    self.bn = nn.BatchNorm2d(output_channels)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.relu(self.bn(self.conv(x)))
    return out
###################### Modelo ###############################
###################### Modelo global ###############################
class GlobalModel(nn.Module):
  def __init__(self,ind=[True,True,True,True]):
    super(GlobalModel, self).__init__()
    self.indicador=ind
    if self.indicador[0]:
      # CNN module for spectrograms
      self.spec_bn = nn.BatchNorm2d(1)
      self.layer1 = Conv_2d(1, 128, pooling=2)
      self.layer2 = Conv_2d(128, 128, pooling=2)
      self.layer3 = Conv_2d(128, 256, pooling=2)
      self.layer4 = Conv_2d(256, 256, pooling=2)
      self.layer5 = Conv_2d(256, 256, pooling=2)
      self.layer6 = Conv_2d(256, 256, pooling=2)
      self.layer7 = Conv_2d(256, 512, pooling=2)
      self.layer8 = Conv_emb(512, 200)
    if self.indicador[1]:
      # FC module for collaborative filtering embedding
      self.cf_fc1 = nn.Linear(200, 512)
      self.cf_bn1 = nn.BatchNorm1d(512)
      self.cf_fc2 = nn.Linear(512, 200)
    if self.indicador[2]:
      # edit module for cover embedding
      self.edit_fc1 = nn.Linear(2048, 1024)
      self.edit_bn1 = nn.BatchNorm1d(1024)
      self.edit_fc2 = nn.Linear(1024, 512)
      self.edit_bn2 = nn.BatchNorm1d(512)
      self.edit_fc3 = nn.Linear(512, 200)
    if self.indicador[3]:
      # acuoustic numeric module
      self.numeric_fc1=nn.Linear(52,100)
      self.numeric_bn1=nn.BatchNorm1d(100)
      self.numeric_fc2=nn.Linear(100,200)

    # FC module for concatenated embedding
    input_shape=sum(ind)*200
    self.cat_fc1 = nn.Linear(input_shape, 512)
    self.cat_bn1 = nn.BatchNorm1d(512)
    self.cat_fc2 = nn.Linear(512, 256)

    # FC module for word embedding
    self.fc1 = nn.Linear(300, 512)
    self.bn1 = nn.BatchNorm1d(512)
    self.fc2 = nn.Linear(512, 256)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.5)

  def spec_to_embedding(self, spec):
    out = spec.unsqueeze(1)
    out = self.spec_bn(out)

    # CNN
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = self.layer6(out)
    out = self.layer7(out)
    out = self.layer8(out)
    out = out.squeeze(2)
    out = nn.MaxPool1d(out.size(-1))(out)
    out = out.view(out.size(0), -1)
    return out

  def cf_to_embedding(self, cf):
    out = self.cf_fc1(cf)
    out = self.cf_bn1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.cf_fc2(out)
    return out
  def cover_to_embedding(self, cover):
    out = self.edit_fc1(cover)
    out = self.edit_bn1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.edit_fc2(out)
    out = self.edit_bn2(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.edit_fc3(out)
    return out
  def numeric_to_embedding(self, ind_numeric):
    out = self.numeric_fc1(ind_numeric)
    out = self.numeric_bn1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.numeric_fc2(out)
    return out

  def cat_to_embedding(self, cat):

    out = self.cat_fc1(cat)
    out = self.cat_bn1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.cat_fc2(out)
    return out

  def song_to_embedding(self, spec, cf,cover,ind_numeric):
    # spec to embedding
    out_cat=torch.tensor(np.array([],dtype=np.float32)).cuda()
    if self.indicador[0]:
      out_spec = self.spec_to_embedding(spec)
      out_cat=torch.cat([out_cat,out_spec],-1)
    # cf to embedding
    if self.indicador[1]:
      out_cf = self.cf_to_embedding(cf)
      out_cat=torch.cat([out_cat,out_cf],-1)
    if self.indicador[2]:
      out_cover = self.cover_to_embedding(cover)
      out_cat=torch.cat([out_cat,out_cover],-1)
    if self.indicador[3]:
      out_numeric = self.numeric_to_embedding(ind_numeric)
      out_cat=torch.cat([out_cat,out_numeric],-1)


    # fully connected
    out = self.cat_to_embedding(out_cat)
    return out

  def word_to_embedding(self, emb):
    out = self.fc1(emb)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.fc2(out)
    return out

  def forward(self, tag, spec, cf, cover, ind_numeric):

    tag_emb = self.word_to_embedding(tag)
    song_emb = self.song_to_embedding(spec, cf,cover,ind_numeric)
    return tag_emb, song_emb


In [15]:
def lastfm_get(user,api_key,artist,track):
    metodo='track.getTopTags'
    #api_secret_key='007f573030c1d37291515f0afc242553'
    user_agent='Dataquest'
    # define headers and URL
    headers = {'user-agent': 'Dataquest'}
    url = 'http://ws.audioscrobbler.com/2.0/'
    # Add API key and format to the payload
    payload={
            'user':user,
            'api_key':api_key,
             'method': 'track.getTopTags',
            'format':'json',
            'artist':artist,
            'track':track
    }


    response = requests.get(url, headers=headers, params=payload)
    return response.json()


def preprocess_tags(tags,cultural_tags):
    #matriz con ceros que se ira llenando con la cantidad de tags asociados a la canción

    df=pd.DataFrame(0,columns=['conteo'],index= sorted(cultural_tags))


    for tag_0 in tags.keys():
        tag=str(tag_0).lower()
        tag= re.sub('[.,&]','',tag)
        tag= re.sub('musica','',tag)
        tag= re.sub('music','',tag)
        tag= re.sub('-',' ',tag)
        tag= re.sub('   ',' ',tag)
        tag= re.sub('  ',' ',tag)
        tag= re.sub('songs','',tag)
        tag= re.sub('song','',tag)
        tag= re.sub('tag','',tag)
        tag= re.sub('\'s','',tag)
        tag= re.sub('0s','0d',tag)
        tag= re.sub('000','0',tag)
        tag= re.sub('111','1',tag)
        tag= re.sub('222','2',tag)
        tag= re.sub('333','3',tag)
        tag= re.sub('444','4',tag)
        tag= re.sub('555','5',tag)
        tag= re.sub('666','6',tag)
        tag= re.sub('777','7',tag)
        tag= re.sub('888','8',tag)
        tag= re.sub('999','9',tag)
        tag= re.sub('sss','s',tag)
        # if we found the tag in cultural tags we add
        if  tag in cultural_tags:
            df.loc[tag,'conteo']=tags[tag_0]
        else:
            for tag_1 in cultural_tags:
                if tag in tag_1:
                    df.loc[tag_1,'conteo']=tags[tag_0]
                else:
                    if tag_1 in tag:
                          df.loc[tag_1,'conteo']=tags[tag_0]
    return df[df.conteo>0].sort_index()

def get_numeric(file,var_names):
    v_numeric=pd.DataFrame(np.zeros((1,len(var_names))),columns=var_names)
    get_dance=Danceability()
    get_energy=Energy()
    FD=FadeDetection()
    DC=DynamicComplexity()
    audio = essentia.standard.MonoLoader(filename=file)()
    _,_,bpm,_,_,fp_bpm,fp_spread,fp_w,sp_bpm,sp_spread,sp_w,_=RhythmDescriptors()(audio)
    v_numeric.loc[0,'bpm']=bpm
    v_numeric.loc[0,'fp_bpm']=fp_bpm
    v_numeric.loc[0,'fp_spread']=fp_spread
    v_numeric.loc[0,'fp_w']=fp_w
    v_numeric.loc[0,'sp_bpm']=sp_bpm
    v_numeric.loc[0,'sp_spread']=sp_spread
    v_numeric.loc[0,'sp_w']=sp_w

    v_numeric.loc[0,'danceability'],_=get_dance(audio)
    v_numeric.loc[0,'energy']=get_energy(audio)
    chor_changes_rate,_,chords_key,chords_number_rate,_,chords_scale,chords_strength,_,_,key_key,key_scale,key_strength=TonalExtractor()(audio)
    v_numeric.loc[0,'chor_changes_rate']=chor_changes_rate
    v_numeric.loc[0,'chor_number_rate']=chords_number_rate
    v_numeric.loc[0,'key_strength']=key_strength
    v_numeric.loc[0,'chords_strength_n']=len(chords_strength)
    if len(chords_strength)>0:
      chords_strength_mean=pd.DataFrame(chords_strength).mean()[0]
      chords_strength_sd=pd.DataFrame(chords_strength).std()[0]
      chords_strength_skew=pd.DataFrame(chords_strength).skew()[0]
      chords_strength_curt=pd.DataFrame(chords_strength).kurtosis()[0]
    else:
      chords_strength_mean=0
      chords_strength_sd=0
      chords_strength_skew=0
      chords_strength_curt=0
    v_numeric.loc[0,'chords_strength_mean']=chords_strength_mean
    v_numeric.loc[0,'chords_strength_sd']=chords_strength_sd
    v_numeric.loc[0,'chords_strength_skew']=chords_strength_skew
    v_numeric.loc[0,'chords_strength_kurtosis']=chords_strength_curt
    fade_in,fade_out=FD(audio)
    if fade_in.shape[0]>0:
      fade_time1=pd.DataFrame(fade_in[:,1]-fade_in[:,0])
      fade_mean1=fade_time1.mean()[0]
      fade_sd1=fade_time1.std()[0]
      fade_skew1=fade_time1.skew()[0]
      fade_kurt1=fade_time1.kurtosis()[0]
    else:
      fade_mean1=0
      fade_sd1=0
      fade_skew1=0
      fade_kurt1=0
    if fade_out.shape[0]>0:
      fade_time2=pd.DataFrame(fade_out[:,1]-fade_out[:,0])
      fade_mean2=fade_time2.mean()[0]
      fade_sd2=fade_time2.std()[0]
      fade_skew2=fade_time2.skew()[0]
      fade_kurt2=fade_time2.kurtosis()[0]
    else:
      fade_mean2=0
      fade_sd2=0
      fade_skew2=0
      fade_kurt2=0
    v_numeric.loc[0,'fade_mean1']=fade_mean1
    v_numeric.loc[0,'fade_mean2']=fade_mean2
    v_numeric.loc[0,'fade_sd1']=fade_sd1
    v_numeric.loc[0,'fade_sd2']=fade_sd2
    v_numeric.loc[0,'fade_skew1']=fade_skew1
    v_numeric.loc[0,'fade_skew2']=fade_skew2
    v_numeric.loc[0,'fade_kurt1']=fade_kurt1
    v_numeric.loc[0,'fade_kurt2']=fade_kurt2

    v_numeric.loc[0,'dynamicComplexity'],v_numeric.loc[0,'loudness']=DC(audio)
    v_numeric.loc[0,'intensity']=Intensity()(audio)
    if chords_scale=='minor':
      v_numeric.loc[0,'chord_minor']=1
    if key_scale=='minor':
      v_numeric.loc[0,'key_minor']=1
    if chords_key!='A':
      x='chord_'+chords_key
      v_numeric.loc[0,x]=1
    if key_key!='A':
      x='key_'+key_key
      v_numeric.loc[0,x]=1
    return v_numeric

def predict_cf(df,X):
    df=df+1
    n=df.shape[0]
    p=np.array([1]*n,ndmin=2).T
    C=np.diag(df.conteo.to_list())
    aux1=(X.values)@C
    aux2=self.xx+aux1@(X.T.values)+np.eye(200)
    term1=np.linalg.inv(aux2)
    Y=np.reshape(term1@aux1@p,200)
    return Y



##Clase recuperador

In [16]:
class recuperador():
  def __init__(self,user, api_key):
    self.user=user
    self.api_key=api_key

    #Model to make the avaluation of the Input
    self.model=GlobalModel()
    S = torch.load('/content/drive/MyDrive/Codigos Tesis Tulio/Modelos/last_1111.ckpt',map_location='cpu')['state_dict']
    SS = {key[6:]: S[key] for key in S.keys()}
    self.model.load_state_dict(SS)
    self.model.eval()

    # auxuliar data for our process
    #general
    self.raw_tracks=pd.read_csv('/content/drive/MyDrive/Codigos Tesis Tulio/raw_tracks.csv',index_col=0)
    self.cat_embs=pkl.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/embeddings/1111_embeddings.pkl','rb'))
    self.spec_embs=None
    self.numeric_embs=None
    self.cover_embs=None
    self.result=None
    self.cf_embs=pkl.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/embeddings/1111_cf_embeddings.pkl','rb'))


    #Cultural
    self.conf_tags=pkl.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/embeddings/cultural_tags_conf.pkl','rb'))
    self.xx=pkl.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/embeddings/cultural_tags_aux.pkl','rb'))
   #Acoustic
    self.numeric_sd=np.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/n_var.npy','rb'))
    self.numeric_mean=np.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/n_mean.npy','rb'))
    self.var_names=pd.read_csv('/content/drive/MyDrive/Codigos Tesis Tulio/splits/indicadores.csv',index_col=0).columns


  ################ cultural information to   embeddings##############################################

  def get_cf_emb(self,track,artist):
    r=lastfm_get(self.user,self.api_key,artist,track)
    if len(r)==1:
      n_tags=len(r['toptags']['tag'])
            #Si hay al menos un tag
      if n_tags>0:
        tags={}

        for tag in r['toptags']['tag']:
          tags[tag['name']]=tag['count']
        df=preprocess_tags(tags,[str(x) for x in self.conf_tags.keys()])

        X=dict(filter(lambda x: x[0] in df.index,self.conf_tags.items()))
        X=pd.DataFrame(X)

        y=predict_cf(df,X)
        Y=torch.tensor(np.reshape(y.astype('float32'),(1,len(y))))
        cf_emb=self.model.cf_to_embedding(Y)
        return cf_emb
      else:
        print('Did not find tags')
    else:
      print('Did not find track')
  ################  editorial information    to embeddings##############################################
  def get_cover_emb(self,image):
    im = cv2.imread(image)
    im_resized = cv2.resize(im, (300, 300), interpolation=cv2.INTER_LINEAR)
    pre_emb=self.transfer_model(im_resized.reshape((1,300,300,3)))
    a=layers.MaxPooling2D((10, 10))(pre_emb)
    a=layers.Flatten()(a)
    a=torch.tensor(np.array(a))
    cover_emb=self.model.cover_to_embedding(a)
    return cover_emb

  ############### acustic information ###################################
  def get_spec_emb(self,signal):
    y, _ = librosa.load(signal)
    spec=librosa.feature.melspectrogram(y=y,sr=22050,n_fft=1024,hop_length=512,n_mels=128)
    self.consult_spec=spec
    if spec.shape[1] < 173:
      nspec = np.zeros((128, self.input_length))
      nspec[:, :spec.shape[1]] = spec
      spec = nspec
    num_chunk=8
    hop = (spec.shape[1] - 173) // num_chunk
    spec = np.array([spec[:, i*hop:i*hop+173] for i in range(num_chunk)])
    spec=torch.tensor(spec)
    out = self.model.spec_to_embedding(spec)
    spec_emb = out.mean(dim=0)
    return spec_emb

  def get_numeric_emb(self,signal):
    numeric=get_numeric(signal,self.var_names)
    numeric=(numeric-self.numeric_mean)/self.numeric_sd
    numeric=torch.tensor(np.array(numeric).astype('float32'))
    numeric_emb=self.model.numeric_to_embedding(numeric)
    return numeric_emb

### Get the anchor track
  def get_anchor(self,embedding,set_embeddings):
    maximo=-1
    arg_maximo=0

    for id_fma,emb in set_embeddings.items():
      sim=nn.CosineSimilarity(dim=-1)(emb, embedding)
      if sim>maximo:
        maximo=sim
        arg_maximo=id_fma
    return arg_maximo,maximo
### Get anchor's neighbors
  def get_neighbors(self,anchor):
      embedding=self.cat_embs[anchor]
      fma_ids=np.array(list(self.cat_embs.keys()))
      embeddings=[np.array(val) for val in self.cat_embs.values()]
      sim=nn.CosineSimilarity(dim=-1)(embedding, torch.tensor(np.array(embeddings)))
      sorted_ids=list(np.argsort(np.array(-sim)))
      return fma_ids[sorted_ids]

##Make the recomendations
  def get_recomendations(self):
      ids=[]
      artistas_id=[]
      results={}
      contador=0
      indice=0
      if len(self.aux_vars)>0:
        while contador<self.k:
          for key in sorted(self.aux_vars.keys()):
            if self.aux_vars[key][indice] not in ids and self.aux_vars[key][indice] in self.raw_tracks.index:
              id=self.aux_vars[key][indice]
              if self.raw_tracks.loc[id,'artist_id'] not in artistas_id: #Only one song for artist
                ids.append(id)
                artistas_id.append(self.raw_tracks.loc[id,'artist_id'])
                meta_data={'artist':self.raw_tracks.loc[id,'artist_name'],
                          'album':self.raw_tracks.loc[id,'album_title'],
                          'track':self.raw_tracks.loc[id,'track_title'],
                          'genres':self.raw_tracks.loc[id,'track_genres'],
                          'path':'/content/drive/MyDrive/Codigos Tesis Tulio/Subconjunto/'+str(id).zfill(6)+'.mp3'
                          }
                results[contador]=meta_data
                contador+=1
            if contador==self.k:
              break
          indice+=1
      return results
  #Query given a Input mode
  def query(self,signal=None,image=None,track=' ',artist=' ',k=10):

      ''' signal: is a path that contains an audio file
          image: is a path that contains an image file
          track: is the name of a track
          artist: is the name of a artist
          k: is the number of recomendations

      '''

      self.k=k
      self.aux_vars={}



      if signal is not None:
        if self.spec_embs is None:
          self.spec_embs=pkl.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/embeddings/1111_spec_embeddings.pkl','rb'))
          self.numeric_embs=pkl.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/embeddings/1111_numeric_embeddings.pkl','rb'))
        spec_emb=self.get_spec_emb(signal)
        numeric_emb=self.get_numeric_emb(signal)
        spec_anchor,spec_distance=self.get_anchor(spec_emb,self.spec_embs)
        numeric_anchor,numeric_distance=self.get_anchor(numeric_emb,self.spec_embs)

        self.aux_vars[spec_distance]=self.get_neighbors(spec_anchor)
        self.aux_vars[numeric_distance]=self.get_neighbors(numeric_anchor)


      if image is not None:
        if self.cover_embs is None:
          self.cover_embs=pkl.load(open('/content/drive/MyDrive/Codigos Tesis Tulio/embeddings/1111_cover_embeddings.pkl','rb'))
          self.transfer_model=transfer.ResNet101(weights='imagenet',include_top=False,input_shape=(300, 300,3))
        cover_emb=self.get_cover_emb(image)

        cover_anchor,cover_distance=self.get_anchor(cover_emb,self.cover_embs)
        self.aux_vars[cover_distance]=self.get_neighbors(cover_anchor)


      cf_emb=self.get_cf_emb(track=track,artist=artist)
      if cf_emb is not None:
        cf_anchor,cf_distance=self.get_anchor(cf_emb,self.cf_embs)
        self.aux_vars[cf_distance]=self.get_neighbors(cf_anchor)

      self.result=self.get_recomendations()



      return self.result


### Show the track resulting
  def show_track(self,ranking=0):
    '''
    rating is the ranking of recommendered songs to be showed
    '''

    if self.result:
      meta_data=self.result[ranking]
      print('artist: ',meta_data['artist'])
      print('album: ',meta_data['album'])
      print('track: ',meta_data['track'])
      return IPython.display.Audio(meta_data['path'])

    else:
      print('make the query method first')







## Test

In [17]:
user='tulio_93'
api_key='e5874997368a1064232683ec69d610f3'
## Create the instanse
sys_rec=recuperador(user,api_key)

<ipython-input-16-5c809fba341f>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  S = torch.load('/content/drive/MyDrive/Codigos Tesis Tulio/Modelos/last_1111.ckpt',map_locat

In [18]:
#Make the query, audio and image files
result=sys_rec.query(signal='/content/drive/MyDrive/Codigos Tesis Tulio/Pruebas/A Wolf at the Door.mp3',
                     image='/content/drive/MyDrive/Codigos Tesis Tulio/Pruebas/radiohead_2003.jpg')

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Did not find track


In [21]:
### listen the recommendation number x
sys_rec.show_track(5)

artist:  Juan Antonio Nieto
album:  Test
track:  test-6º AERIAL
